In [ ]:
# 패키지 설치
!pip install pickle5 --quiet

In [ ]:
# 패키지 임포트
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import pickle
sns.set()

# Momentum Factor 전략

In [ ]:
# 가격 데이터 로드
with open('equity_price.pkl', 'rb') as f:
    price = pickle.load(f)

In [ ]:
price

In [ ]:
price.pct_change().cumsum()

In [ ]:
# 주식 모멘텀 클래스
class EquityMomentum:

    # 초기화 함수
    def __init__(self, price, lookback_period, n_selection, cost=0.0005, signal_method='dm', long_only=False):

        # 수익률
        self.rets = price.pct_change().fillna(0)

        # 모멘텀 방식 선택
        if signal_method == 'am':
            self.signal = self.absolute_momentum(price, lookback_period, long_only)
        elif signal_method == 'rm':
            self.signal = self.relative_momentum(price, lookback_period, n_selection, long_only)
        elif signal_method == 'dm':
            self.signal = self.dual_momentum(price, lookback_period, n_selection, long_only)

        # 거래비용
        self.cost = cost
        
        # 포트폴리오 수익률
        self.port_rets = self.calculate_returns(self.rets, self.signal, self.cost)

        # 샤프비율
        self.sharpe_ratio = self.calculate_sharpe_ratio(self.port_rets)
        
        # VaR 대비 성과 비율
        self.VaR_ratio = self.calculate_VaR_ratio(self.port_rets)
        
        # CVaR 대비 성과 비율
        self.CVaR_ratio = self.calculate_CVaR_ratio(self.port_rets)
        
        # 승률
        self.hit_ratio = self.calculate_hit_ratio(self.port_rets)
        
        # 손익비
        self.gtp_ratio = self.calculate_gtp_ratio(self.port_rets)
        
        # 백테스팅 결과
        self.plot_result(self.port_rets)

    # 절대 모멘텀 시그널 계산 함수
    def absolute_momentum(self, price, lookback, long_only=False):

        # N일 수익률  
        returns = price.pct_change(periods=lookback)

        # 롱 시그널
        long_signal = (returns > 0) * 1

        # 숏 시그널
        short_signal = (returns < 0) * -1

        # 토탈 시그널
        if long_only:
            signal = long_signal
        else:
            signal = long_signal + short_signal
        
        return signal
    
    # 상대 모멘텀 시그널 계산 함수
    def relative_momentum(self, price, lookback, n_selection, long_only=False):

        # N일 수익률
        returns = price.pct_change(periods=lookback)

        # 수익률 순위화
        rank = returns.rank(axis=1, ascending=False)

        # 롱 시그널
        long_signal = (rank <= n_selection) * 1

        # 숏 시그널
        short_signal = (rank >= len(rank.columns) - n_selection + 1) * -1

        # 토탈 시그널
        if long_only:
            signal = long_signal
        else:
            signal = long_signal + short_signal

        return signal
    
    # 듀얼 모멘텀 시그널 계산 함수
    def dual_momentum(self, price, lookback, n_selection, long_only=False):

        # 절대 모멘텀 시그널
        abs_signal = self.absolute_momentum(price, lookback, long_only)

        # 상대 모멘텀 시그널
        rel_signal = self.relative_momentum(price, lookback, n_selection, long_only)

        # 듀얼 모멘텀 시그널
        signal = (abs_signal == rel_signal) * abs_signal

        return signal

    # 포트폴리오 수익률 계산 함수
    def calculate_returns(self, rets, signals, cost):

        # 포트폴리오 수익률
        port_rets = signals.shift() * rets - abs(signals.diff()) * cost

        return port_rets.mean(axis=1)

    # 샤프비율 계산
    def calculate_sharpe_ratio(self, total_returns):
        sharpe_ratio = total_returns.mean() * np.sqrt(252) / total_returns.std()

        return sharpe_ratio
    
    # VaR 대비 성과비율 계산
    def calculate_VaR_ratio(self, total_returns, delta=0.01):
        VaR = total_returns.quantile(delta)
        VaR_ratio = - total_returns.mean() / VaR
        
        return VaR_ratio
    
    # CVaR 대비 성과비율 계산
    def calculate_CVaR_ratio(self, total_returns, delta=0.01):
        def calculate_CVaR(rets, delta):
            VaR = rets.quantile(delta)
            return rets[rets <= VaR].mean()
        
        CVaR = calculate_CVaR(total_returns, delta)
        CVaR_ratio = - total_returns.mean() / CVaR
        
        return CVaR_ratio
    
    # 승률 계산 함수
    def calculate_hit_ratio(self, total_returns):
        return len(total_returns[total_returns > 0.0]) / len(total_returns[total_returns != 0.0])

    # 손익비 계산 함수
    def calculate_gtp_ratio(self, total_returns):
        return total_returns[total_returns > 0.0].mean() / -total_returns[total_returns < 0.0].mean()
    
    # 백테스팅 결과 시각화 함수
    def plot_result(self, rets):

        # 백테스팅 결과 시각화
        plt.figure(figsize=(12, 6))
        rets.cumsum().plot()
        plt.show()

### 파라미터 세팅

In [ ]:
# 룩백 윈도우
lookback = 20 * 24

# 상대 모멘텀 롱숏 개수
n_selection = len(price.columns) / 4

In [ ]:
# 주식 모멘텀 백테스팅
momentum = EquityMomentum(price, lookback, n_selection, signal_method='dm', long_only=True)

In [ ]:
momentum.port_rets.cumsum()

In [ ]:
((1+momentum.port_rets).cumprod()-1)

In [ ]:
momentum.sharpe_ratio, momentum.VaR_ratio, momentum.CVaR_ratio

In [ ]:
# 확률적 우위 검증
momentum_TE = momentum.gtp_ratio > (1 - momentum.hit_ratio) / momentum.hit_ratio
momentum.gtp_ratio, momentum.hit_ratio, momentum_TE


# Value Factor 전략

In [ ]:
# 데이터 로드
# 가격 데이터
with open('commodity_price.pkl', 'rb') as f:
    price = pickle.load(f)

# CFTC Hedge 롱포지션
with open('long_pos.pkl', 'rb') as f:
    long_pos = pickle.load(f)

# CFTC Hedge 숏포지션
with open('short_pos.pkl', 'rb') as f:
    short_pos = pickle.load(f)

# CFTC 전체 포지션
with open('total_pos.pkl', 'rb') as f:
    total_pos = pickle.load(f)

In [ ]:
price, long_pos, short_pos, total_pos

In [ ]:
# 원자재 밸류 팩터 전략 구현 클래스
class CommodityValue:
    
    # 초기화 함수
    def __init__(self, price, long_pos, short_pos, total_pos):
        
        # 가격 데이터
        self.price = price
        
        # 포지션 데이터
        # CFTC Hedger 롱포지션
        self.long_pos = long_pos
        
        # CFTC Hedger 숏포지션
        self.short_pos = short_pos
        
        # CFTC 전체포지션
        self.total_pos = total_pos

        # 롱숏 개수
        self.n = int(len(self.price.columns) / 2)
        
        # 룩백 윈도우
        self.lookback = 52
        
        # 투자 가중치
        self.weights = self.calculate_weights(self.long_pos, self.short_pos, self.total_pos, self.lookback)
        
        # 거래비용
        self.cost = 0.0005
                
        # 포트폴리오 자산별 수익률
        self.port_rets = self.calculate_returns(self.price, self.weights, self.cost)
        self.port_rets.dropna(inplace=True)
        
        # 샤프비율
        self.sharpe_ratio = self.calculate_sharpe_ratio(self.port_rets)
        
        # VaR 대비 성과 비율
        self.VaR_ratio = self.calculate_VaR_ratio(self.port_rets)
        
        # CVaR 대비 성과 비율
        self.CVaR_ratio = self.calculate_CVaR_ratio(self.port_rets)
        
        # 승률
        self.hit_ratio = self.calculate_hit_ratio(self.port_rets)
        
        # 손익비
        self.gtp_ratio = self.calculate_gtp_ratio(self.port_rets)
        
        # 포트폴리오 백테스팅 결과 시각화
        self.plot_port_returns(self.port_rets)
    
    # 투자 가중치 계산 함수
    def calculate_weights(self, long_pos, short_pos, total_pos, lookback=52):
        
        # 롱숏 헤징 레인지
        long_hedging = long_pos / total_pos
        short_hedging = short_pos / total_pos

        long_hedging_range = (long_hedging - long_hedging.rolling(lookback).min()) / (long_hedging.rolling(lookback).max() - long_hedging.rolling(lookback).min())
        short_hedging_range = (short_hedging - short_hedging.rolling(lookback).min()) / (short_hedging.rolling(lookback).max() - short_hedging.rolling(lookback).min())
                                
        # 롱숏 랭크
        long_rank = long_hedging_range.rank(axis=1)
        short_rank = short_hedging_range.rank(axis=1)
        
        # 투자 가중치
        long_weights = (long_rank > self.n) * (1/self.n)
        short_weights = (short_rank > self.n) * -(1/self.n)
        total_weights = long_weights + short_weights

        return total_weights

    # 수익률 계산
    def calculate_returns(self, price, weights, cost):
        
        # 일별 수익률
        rets = price.pct_change()

        # 가중치
        weights = weights.reindex_like(price)
        weights.ffill(inplace=True)

        # 포트폴리오 수익률
        port_rets = (weights.shift() * rets - abs(weights.diff()) * cost)

        return port_rets.sum(axis=1)
    
    # 샤프비율 계산
    def calculate_sharpe_ratio(self, total_returns):
        sharpe_ratio = total_returns.mean() * np.sqrt(252) / total_returns.std()
        return sharpe_ratio
    
    # VaR 대비 성과비율 계산
    def calculate_VaR_ratio(self, total_returns, delta=0.01):
        VaR = total_returns.quantile(delta)
        VaR_ratio = - total_returns.mean() / VaR
        
        return VaR_ratio
    
    # CVaR 대비 성과비율 계산
    def calculate_CVaR_ratio(self, total_returns, delta=0.01):
        def calculate_CVaR(rets, delta):
            VaR = rets.quantile(delta)
            return rets[rets <= VaR].mean()
        
        CVaR = calculate_CVaR(total_returns, delta)
        CVaR_ratio = - total_returns.mean() / CVaR
        
        return CVaR_ratio
    
    # 승률 계산 함수
    def calculate_hit_ratio(self, total_returns):
        return len(total_returns[total_returns > 0.0]) / len(total_returns[total_returns != 0.0])

    # 손익비 계산 함수
    def calculate_gtp_ratio(self, total_returns):
        return total_returns[total_returns > 0.0].mean() / -total_returns[total_returns < 0.0].mean()
    
    # 백테스팅 결과 시각화
    def plot_port_returns(self, rets):
        plt.figure(figsize=(12, 6))
        rets.cumsum().plot(label='port')
        plt.show()

In [ ]:
# 백테스팅 실행
value = CommodityValue(price, long_pos, short_pos, total_pos)

In [ ]:
value.weights

In [ ]:
value.sharpe_ratio, value.VaR_ratio, value.CVaR_ratio

In [ ]:
value_TE = value.gtp_ratio > (1 - value.hit_ratio) / value.hit_ratio
value.gtp_ratio, value.hit_ratio, value_TE

# Carry Factor 전략

In [ ]:
# 데이터 로드
with open('vol_price.pkl', 'rb') as f:
    df = pickle.load(f)

In [ ]:
# VIX 선물 가격 데이터
price = df.iloc[:, 0]

# VIX 기간구조 기울기 데이터
slope = df.iloc[:, 1]

In [ ]:
slope.plot()

In [ ]:
# 변동성 캐리 전략 구현 클래스
class VolatilityCarry:

    # 초기화 함수
    def __init__(self, price, slope):

        # 변동성 선물 가격
        self.price = price

        # 수익률
        self.rets = self.price.pct_change()

        # 기간구조 기울기
        self.slope = slope

        # 거래비용
        self.cost = 0.002

        # 가중치
        self.weights = self.calculate_weights(self.slope)

        # 포트폴리오 수익률
        self.port_rets = self.calculate_returns(self.rets, self.weights, self.cost)

        # 백테스팅 결과 시각화
        self.plot_port_returns(self.port_rets)

        # 샤프비율
        self.sharpe_ratio = self.calculate_sharpe_ratio(self.port_rets)
        
        # VaR 대비 성과 비율
        self.VaR_ratio = self.calculate_VaR_ratio(self.port_rets)
        
        # CVaR 대비 성과 비율
        self.CVaR_ratio = self.calculate_CVaR_ratio(self.port_rets)
        
        # 승률
        self.hit_ratio = self.calculate_hit_ratio(self.port_rets)
        
        # 손익비
        self.gtp_ratio = self.calculate_gtp_ratio(self.port_rets)

    # 가중치 계산 함수
    def calculate_weights(self, slope):
        
        # 롱 포지션
        long_weights = (slope < 1) * 1

        # 숏 포지션
        short_weights = (slope > 1) * -1

        # 토탈 포지션
        total_weights = long_weights + short_weights

        return total_weights


    # 포트폴리오 수익률 계산
    def calculate_returns(self, rets, weights, cost):
        port_rets = weights.shift() * rets - abs(weights.diff()) * cost

        return port_rets


    # 샤프비율 계산
    def calculate_sharpe_ratio(self, total_returns):
        sharpe_ratio = total_returns.mean() * np.sqrt(252) / total_returns.std()
        return sharpe_ratio
    
    # VaR 대비 성과비율 계산
    def calculate_VaR_ratio(self, total_returns, delta=0.01):
        VaR = total_returns.quantile(delta)
        VaR_ratio = - total_returns.mean() / VaR
        
        return VaR_ratio
    
    # CVaR 대비 성과비율 계산
    def calculate_CVaR_ratio(self, total_returns, delta=0.01):
        def calculate_CVaR(rets, delta):
            VaR = rets.quantile(delta)
            return rets[rets <= VaR].mean()
        
        CVaR = calculate_CVaR(total_returns, delta)
        CVaR_ratio = - total_returns.mean() / CVaR
        
        return CVaR_ratio
    
    # 승률 계산 함수
    def calculate_hit_ratio(self, total_returns):
        return len(total_returns[total_returns > 0.0]) / len(total_returns[total_returns != 0.0])

    # 손익비 계산 함수
    def calculate_gtp_ratio(self, total_returns):
        return total_returns[total_returns > 0.0].mean() / -total_returns[total_returns < 0.0].mean()

    # 백테스팅 결과 시각화
    def plot_port_returns(self, rets):
        plt.figure(figsize=(12, 6))
        rets.cumsum().plot(label='port')
        plt.show()

In [ ]:
carry = VolatilityCarry(price, slope)

In [ ]:
carry.sharpe_ratio, carry.VaR_ratio, carry.CVaR_ratio

In [ ]:
carry_TE = carry.gtp_ratio > (1 - carry.hit_ratio) / carry.hit_ratio
carry.gtp_ratio, carry.hit_ratio, carry_TE